In [59]:
import pandas as pd
import os
import glob
import csv
import requests 
from pprint import pprint

In [13]:

for i in range(16):
    path = os.path.join(r"../rawdata/iahhist_mf",f"'modified_df_{i}'")
    print(path)

../rawdata/iahhist_mf\'modified_df_0'
../rawdata/iahhist_mf\'modified_df_1'
../rawdata/iahhist_mf\'modified_df_2'
../rawdata/iahhist_mf\'modified_df_3'
../rawdata/iahhist_mf\'modified_df_4'
../rawdata/iahhist_mf\'modified_df_5'
../rawdata/iahhist_mf\'modified_df_6'
../rawdata/iahhist_mf\'modified_df_7'
../rawdata/iahhist_mf\'modified_df_8'
../rawdata/iahhist_mf\'modified_df_9'
../rawdata/iahhist_mf\'modified_df_10'
../rawdata/iahhist_mf\'modified_df_11'
../rawdata/iahhist_mf\'modified_df_12'
../rawdata/iahhist_mf\'modified_df_13'
../rawdata/iahhist_mf\'modified_df_14'
../rawdata/iahhist_mf\'modified_df_15'


In [14]:
# %%
# IMPORT MODULES
import pandas as pd
import os
import glob
import csv

# %%
# IMPORT FILE PATHS 
active_project = "../rawdata/iahhist_mf/"

file_paths = glob.glob(os.path.join(active_project, "*.csv"))

# %%
# setting up the list for all raw dfs
all_df = []

# listing turning movement counts as current 
current_csv_list = []

for file in file_paths:
    current_csv_list.append(file)

# %%
# function to read all csvs listed 

def read_csv(file_path):
    try:
        imp_df = pd.read_csv(file_path, 
                             sep='delimiter',
                             header=None,
                             engine='python'
                            )
        return imp_df #imp_df as in imported dataframe
    except pd.errors.ParserError as e:
        return (file_path, f"Error parsing CSV: {str(e)}")  # Return tuple with file path and error message
    except FileNotFoundError:
        return (file_path, "File not found")  # Return tuple with file path and error message

# %%
## validating dataframes - sometimes there's errors

for file_path in current_csv_list:
    imp_df = read_csv(file_path)  # Assign the returned DataFrame to imp_df
    all_df.append(imp_df)  # Append imp_df to all_df

valid_dfs = [df for df in all_df if isinstance(df, pd.DataFrame)]  # Filter out non-DataFrame objects

# %%
## modify each dataframe

def modify_in_df(df):
 # modifications to the df go here
    raw_data_startrow = df.index[df.iloc[:, 0].str.startswith("Carrier Code")].tolist()[0]
    raw_data = df.iloc[raw_data_startrow:, :]  # Extract from the starting row onwards
    raw_data.reset_index(drop=True, inplace=True)
    raw_data = raw_data.rename(columns={raw_data.columns[0]:"rawdata"})
    raw_table = raw_data["rawdata"].str.split(',',expand=True) #split columns
    raw_header = raw_table.iloc[0]
    raw_table.columns = raw_header  # Set the new header
    raw_table = raw_table.iloc[1:]  # Remove the first row
    #raw_table = raw_table.apply(lambda x: x.str.strip('"'))
    #raw_table["Date (MM/DD/YYYY)"] = pd.to_datetime(raw_table["Date (MM/DD/YYYY)"])
    #raw_table = raw_table[raw_table["Date (MM/DD/YYYY)"].dt.year > 2012]
    return raw_table 

# %%
## loop through dataframes

modified_df_list = []  # List to store the modified DataFrames
error_log = []  # List to store the error details
df = pd.DataFrame()

for i in range(len(valid_dfs)):
    try:
        df = valid_dfs[i]  # Retrieve the DataFrame by index
        modified_df = modify_in_df(df) # use this step to Modify the DataFrame
        modified_df_list.append(modified_df)  # Append the modified DataFrame to the list
        # Generate CSV file for the modified DataFrame
        # filename = f"modified_df_{i}.csv"
        #modified_df.to_csv(filename, index=False)
    except Exception as e:
        error_log.append((i, str(e)))  # Log the index and error message

# %%
big_df = pd.concat(modified_df_list, ignore_index=True)

historical_iah = big_df


In [71]:
#clean the data 
df = pd.DataFrame(historical_iah)
df_cleaned = df["Destination Airport"].unique()
airports = pd.DataFrame({"Airport Code": df_cleaned})

airports

,Airport Code
0,MIA
1,DFW
2,JFK
3,CLT
4,PHX
...,...
138,SJT
139,JLN
140,DRO
141,GSO


In [40]:
#set up the API 
api_key = input("what is your api key?")

In [90]:

#list for airport name,city and state 
airport_name =[]
airport_city = []
airport_state= []
lat = []
long = []

for x in range(143):
    airport = airports["Airport Code"][x]
    
    url = "https://airport-info.p.rapidapi.com/airport"
    querystring = {"iata": airport}
    headers = {
        "X-RapidAPI-Key": api_key,
		"X-RapidAPI-Host": "airport-info.p.rapidapi.com"
		}

    response = requests.get(url, headers=headers, params=querystring).json()
    airport_name.append(response["name"])
    airport_city.append(response["city"])
    airport_state.append(response["state"])
    lat.append(response["latitude"])
    long.append(response["longitude"])
# airports

In [105]:
#add to the DF 

airports["Airport Name"]= airport_name
airports["City"]= airport_city
airports["State"] = airport_state
airports["Lat"] = lat
airports["Long"]=long
airports["Hotel Name"] = ""
airports["Hotel Address"] = ""
airports["Hotel Distance"] = ""
airports

airports.to_csv("CSV/Airportnames")

In [89]:
geoapify_key = input("what is your geoapify key?")

for index, row in airports.iterrows():
    latitude= airports["lat"][x]
    longitude = airports["long"][x]
    # Set the parameters for the type of search
    limit = 20
    radius = 30
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"

    # set up a parameters dictionary
    params = {
        "filter":filters,
        "limit":limit,
        "bias":bias,
        "apiKey":geoapify_key    
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    

    
    # Get the ethnicity type from the current DataFrame's row
    airport_city = airpots.loc[index, "City"]
    # Add the current ethnicity type to the parameters
    params["categories"] = f"accomodation.hotel"
   
    # Make an API request using the params dictionary
    restaurant = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    restaurant = restaurant.json()
    
    # Grab the first restaurant from the results and store the details in the DataFrame
    try:
        types_df.loc[index, "name"] = restaurant["features"][0]["properties"]["name"]
        types_df.loc[index, "address"] = restaurant["features"][0]["properties"]["address_line2"]
        types_df.loc[index, "distance"] = int(restaurant["features"][0]["properties"]["distance"])
    except (KeyError, IndexError):
        # If no restaurant is found, set the estaurant name as "No restaurant found".
        types_df.loc[index, "name"] = "No restaurant found"
        # Set the distance column value to np.nan to allow sorting values
        types_df.loc[index, "distance"] = np.nan
        
    # Log the search results
    print(f"nearest {types_df.loc[index, 'ethnicity']} restaurant: {types_df.loc[index, 'name']}")


[]

In [101]:
geoapify_key = input("what is your geoapify key?")

latitude= airports["Lat"][2]
longitude = airports["Long"][2]
    # Set the parameters for the type of search
limit = 20
radius = 30
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

# set up a parameters dictionary
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key

    }
params["categories"] = f"accommodation.hotel"
    # Set base URL
base_url = "https://api.geoapify.com/v2/places"
restaurant = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
restaurant = restaurant.json()

In [103]:

print(restaurant)

{'type': 'FeatureCollection', 'features': []}


In [ ]:

#list for airport name,city and state 
airport_name =[]
airport_city = []
airport_state= []
lat = []
long = []

for x in range(143):
    airport = airports["Airport Code"][x]
    
    url = "https://airport-info.p.rapidapi.com/airport"
    querystring = {"iata": airport}
    headers = {
        "X-RapidAPI-Key": api_key,
		"X-RapidAPI-Host": "airport-info.p.rapidapi.com"
		}

    response = requests.get(url, headers=headers, params=querystring).json()
    airport_name.append(response["name"])
    airport_city.append(response["city"])
    airport_state.append(response["state"])
    lat.append(response["latitude"])
    long.append(response["longitude"])
# airports